# Как парсил данные и создавал БД по недобросовестным поставщиквам

## Необходимые библиотеки

In [ ]:
# для работы с локальными файлами
import zipfile
import os
# для работы с XML
import xmltodict
import xml.etree.ElementTree as ET
# для работы с БД
import pandas as pd

## Разархивируем скачанные zip-файлы из удаленной папки

In [ ]:
def unzip_files(source, destination):
    # Проходим по всем файлам в папке с архивами
    for file_name in os.listdir(source):
        if file_name.endswith('.zip'):
            # Составляем полный путь к архиву
            file_path = os.path.join(source, file_name)
            
            # Открываем ZIP-файл
            with ZipFile(file_path, 'r') as zip_ref:
                # Извлекаем все содержимое в целевую папку
                zip_ref.extractall(destination)

                print(f'Файлы из {file_name} были успешно извлечены в {destination}')

In [ ]:
# Путь к 1 папке, где хранятся ZIP-архивы
source_folder = r'D:\Jupyter_Notebook\2024_ИПС_Данные\ftp.zakupki.gov (UnfairSupplier)'

# Путь ко 2 папке, где хранятся ZIP-архивы
source_folder_additional = r'D:\Jupyter_Notebook\2024_ИПС_Данные\ftp.zakupki.gov (UnfairSupplier2022)'

# Путь к папке, куда будут извлечены файлы
destination_folder = r'D:\Jupyter_Notebook\2024_ИПС_Данные\all_non_zip'

In [ ]:
unzip_files(source_folder, destination_folder)
unzip_files(source_folder_additional, destination_folder)

> в итоге имеем 110 392 записей. ВАЖНО - это xml и sig записи (sig - электронная подпись связанная с некоторыми закупками)

In [ ]:
directory_path = r'D:\Jupyter_Notebook\2024_ИПС_Данные\all_non_zip'

xml_count = 0  # Счетчик для файлов XML
other_count = 0  # Счетчик для других файлов (то есть sig)

# Перебираем файлы в указанной директории
for filename in os.listdir(directory_path):
    if filename.endswith('.xml'):
        xml_count += 1
    else:
        other_count += 1

print(f"Количество файлов XML: {xml_count}")
print(f"Количество других файлов: {other_count}")

> Количество файлов XML: 65157  
Количество других файлов: 45235

## Спарсим данные с XML файлов

### 1. Проверим структуру файла

In [ ]:
with open('unfairSupplier_336825-22_336825_for_test_2022.xml', encoding="utf8") as datafile:
    doc_main_test = xmltodict.parse(datafile.read())

> В некоторых файлах, которые были выборочно проверены вручную, структура отличалась. Но учесть все случаи было трудозатратно. Поэтому изначально бралась наиболее "популярная" структура. Она охватила почти все файлы (62 тыс из 65 тыс), поэтому в других не было необходимости 

### 2. Отберем необходимые переменные

In [56]:
# дата публикации записи в рнп
publish_date = doc_main_test['ns2:export']['ns2:unfairSupplier']['publishDate']

In [59]:
# причина занесения в рнп
reason = doc_main_test['ns2:export']['ns2:unfairSupplier']['createReason']

In [74]:
# код СПЗ - Уникальный учетный номер организации - это Уникальный учетный номер организации (СПЗ)
customer_code = doc_main_test['ns2:export']['ns2:unfairSupplier']['customer']['regNum']

In [65]:
# ИНН заказчика
customer_inn = doc_main_test['ns2:export']['ns2:unfairSupplier']['customer']['INN']

In [68]:
# КПП заказчика
customer_kpp = doc_main_test['ns2:export']['ns2:unfairSupplier']['customer']['KPP']

In [72]:
# полное название поставщика
supplier_full_name = doc_main_test['ns2:export']['ns2:unfairSupplier']['unfairSupplier']['fullName']

In [77]:
# тип поставщика
supplier_type = doc_main_test['ns2:export']['ns2:unfairSupplier']['unfairSupplier']['type']

In [80]:
# ИНН поставщика
supplier_inn = doc_main_test['ns2:export']['ns2:unfairSupplier']['unfairSupplier']['inn']

In [81]:
# КПП поставщика
supplier_kpp = doc_main_test['ns2:export']['ns2:unfairSupplier']['unfairSupplier']['kpp']

In [95]:
# Основатели компании-поставщика
# Основателей может быть несколько, поэтому надо отобрать нужные переменные для нескольких основателей
# ИНН и амплуа (то есть значение в компании или конкретно в этой закупке) основателя 
# надо определить сокращения амплуа из исходного файла - там есть переменная name - подробное описание амплуа
founders_inn_role = [list([info_of_founder['inn'], info_of_founder['type']['code']]) for info_of_founder in doc_main_test['ns2:export']['ns2:unfairSupplier']['unfairSupplier']['founders']]

In [99]:
# объект закупки
purchase_object = doc_main_test['ns2:export']['ns2:unfairSupplier']['purchase']['purchaseObjectInfo']

In [103]:
# объект контракта
# конкретно в doc_main_test нет данного параметра, но в других файлах есть. Это связано с особенностью закупок
contract_object = doc_main_test['ns2:export']['ns2:unfairSupplier']['contract']['productInfo']

In [106]:
# валюта цены конракта
contract_price_currency = doc_main_test['ns2:export']['ns2:unfairSupplier']['contract']['currency']['code']

In [109]:
# цена контракта
contract_price_amount = doc_main_test['ns2:export']['ns2:unfairSupplier']['contract']['price']

### 3. Прочитаем все необходимые параметры XML файла, создадим БД (pandas)

In [ ]:
# Путь к директории с файлами
directory_path = r'D:\Jupyter_Notebook\2024_ИПС_Данные\all_non_zip'

# Подготовим список для хранения данных
data = []

# Перебор всех файлов в директории
for filename in os.listdir(directory_path):   
    
    # Проверяем, является ли файл XML файлом
    if filename.endswith('.xml'):
        file_path = os.path.join(directory_path, filename)
        try:
            # Открываем и читаем XML файл
            with open(file_path, encoding="utf8") as datafile:
                doc = xmltodict.parse(datafile.read())
                
                # Извлекаем нужные переменные, используя метод get для избежания KeyError
                publish_date = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('publishDate', None)
                reason = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('createReason', None)
                customer_code = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('customer', {}).get('regNum', None)
                customer_inn = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('customer', {}).get('INN', None)
                customer_kpp = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('customer', {}).get('KPP', None)
                supplier_full_name = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('unfairSupplier', {}).get('fullName', None)
                supplier_type = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('unfairSupplier', {}).get('type', None)
                supplier_inn = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('unfairSupplier', {}).get('inn', None)
                supplier_kpp = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('unfairSupplier', {}).get('kpp', None)
                
                # Добавляем логику для извлечения информации об основателях
                founders_info = doc.get('ns2:export', {})\
                                .get('ns2:unfairSupplier', {})\
                                .get('unfairSupplier', {})\
                                .get('founders', [])

                # Проверяем, является ли founders списком. Если нет, преобразуем в список
                if founders_info is not None and not isinstance(founders_info, list):
                    founders_info = [founders_info]  # Делаем список с одним элементом     
                    
                founders_inn_role = []
                if founders_info:
                    for info_of_founder in founders_info:
                        inn = info_of_founder.get('inn', None)
                        type_code = info_of_founder.get('type', {}).get('code', None) if info_of_founder.get('type') else None
                        founders_inn_role.append([inn, type_code])
                
                # Проверяем, существует ли 'purchase' и не равно ли оно None
                purchase = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('purchase', {})
                if purchase is not None:
                    purchase_object = purchase.get('purchaseObjectInfo', None)
                else:
                    purchase_object = None

                contract_object = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('contract', {}).get('productInfo', None)
                contract_price_currency = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('contract', {}).get('currency', {}).get('code', None)
                contract_price_amount = doc.get('ns2:export', {}).get('ns2:unfairSupplier', {}).get('contract', {}).get('price', None)
                
                # Добавляем извлеченные данные в список
                data.append({'publish_date': publish_date,
                             'reason': reason,
                             'customer_code': customer_code,
                             'customer_inn': customer_inn,
                             'customer_kpp': customer_kpp,
                             'supplier_full_name': supplier_full_name,
                             'supplier_type': supplier_type, 
                             'supplier_inn': supplier_inn,
                             'supplier_kpp': supplier_kpp,
                             'founders_inn_role': founders_inn_role,
                             'purchase_object': purchase_object,
                             'contract_object': contract_object,
                             'contract_price_currency': contract_price_currency, 
                             'contract_price_amount': contract_price_amount
                            })
                
        except Exception as e:
            # В случае ошибки при чтении файла выводим сообщение
            print(f"Ошибка при чтении файла {file_path}: {e}")

# Создаем DataFrame
df = pd.DataFrame(data)

# Сохраняем DataFrame в CSV файл
df.to_csv('output_dataframe.csv', index=False)

> Как обозначал ранее, скрипт не охвывает все 65 157 записей. Поэтому проверяем, где имеем заполненные мастхэв параметры (дата и ИНН)

> Почему дата и ИНН? - Потому что без них нет возможности найти соответствия с финансовыми показателями поставщика из другой базы (это чуть позже)

In [9]:
df[df['publish_date'].notna() & df['supplier_inn'].notna()]

,publish_date_year,publish_date,reason,customer_code,customer_inn,customer_kpp,supplier_full_name,supplier_type,supplier_inn,supplier_kpp,founders_inn_role,purchase_object,contract_object,contract_price_currency,contract_price_amount
143,2014.0,2014-01-21T14:22:11.636,CANCEL_CONTRACT,3.243001e+09,2.901057e+09,290101001.0,"ООО ""Строительные технологии""",U,2901235160,NaN,[],На право заключения гражданско-правового догов...,ремонту крылец педиатрического корпуса по адре...,RUB,990688.90
144,2014.0,2014-01-24T15:47:55.179,CANCEL_CONTRACT,1.572000e+09,6.027088e+09,602701001.0,"ООО ""РОСПРОМТОРГ""",U,6321299175,NaN,[],Размещение заказа на право \nзаключить государ...,выполнение работ по установке недостающих доро...,RUB,13499999.99
145,2014.0,2014-01-24T17:00:36.339,CANCEL_CONTRACT,3.172000e+09,2.225022e+09,222501001.0,"Общество с ограниченной ответственностью ""Гермес""",U,2225097612,222501001.0,[],Поставка продуктов питания (картофель) для нуж...,Поставка продуктов питания (картофель) для нуж...,RUB,248958.56
146,2014.0,2014-01-27T11:59:04.391,CANCEL_CONTRACT,1.731000e+09,7.701904e+09,770101001.0,"ООО ""АктивСтройСервис""",U,7728800929,NaN,[],Выполнение ремонта скатной кровли администрати...,Выполнение работ по ремонту скатной кровли адм...,RUB,2420682.53
147,2014.0,2014-01-27T12:17:33.049,CANCEL_CONTRACT,6.351000e+09,3.908006e+09,390601001.0,"Общество с ограниченной ответственностью ""ПЕГАС""",U,7713754973,771301001.0,[],Поставка бланков «Мореходная книжка» в количес...,"Поставка бланков ""Мореходная книжка"" для нужд ...",RUB,542741.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62910,2018.0,2018-03-01T00:00:00.000,CANCEL_CONTRACT,1.373000e+09,4.003006e+09,400301001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОПТО...",U,4025419633,402501001.0,"[['402571207928', None]]",Выполнение работ по ремонту фасада МФЦ на ул. ...,Выполнение работ по ремонту фасада МФЦ на ул. ...,RUB,416000.00
62911,2018.0,2018-03-01T00:00:00.000,CANCEL_CONTRACT,1.463000e+09,4.826022e+09,482601001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРИУМФ""",U,1215214501,121501001.0,"[['121526353197', None]]",услуги по переносу на базу системы управления ...,услуги по переносу на базу системы управления...,RUB,31250.00
62912,2018.0,2018-02-19T00:00:00.000,WINNER_DEVIATION,8.104000e+09,1.510018e+09,151001001.0,"ООО ""Т-2""",U,5022030921,502201001.0,"[['502200312533', None], ['502206194300', None]]",Консервы: маринады овощные,"Консервы: маринады овощные ""Лук репчатый марин...",RUB,4011464.28
62913,2018.0,2018-02-19T00:00:00.000,WINNER_DEVIATION,3.483005e+09,5.043007e+09,504301001.0,ООО «Биосфера»,U,6234117365,623401001.0,[],"Оказание услуг по проведению дезинсекции, дера...","Услуги по дезинфекции, дезинсекции и дератизац...",RUB,583221.86


> 62 646 записей, где заполнены мастхэв параметры. Будем работать с этим

### 4. Сделаем первичную предобработку данных и создадим рабочую БД

In [ ]:
# Отбираем строки, где даты публикации и ИНН поставщика не равны NaN
filtered_df = df[df['publish_date'].notna() & df['supplier_inn'].notna()]

# Преобразуем столбец с датой публикации в формат datetime
filtered_df['publish_date'] = pd.to_datetime(filtered_df['publish_date'])

# Извлекаем год из столбца с датой публикации - эта переменная необходима для дальнейшего анализа
filtered_df['year'] = filtered_df['publish_date'].dt.year

# Добавляем столбец с годом в начало таблицы
filtered_df.insert(0, 'publish_date_year', filtered_df.pop('year'))

# Сохраняем новый датасет в файл CSV
filtered_df.to_csv('output_dataframe_cleaned.csv', index=False)  # Индекс не сохраняем

In [1]:
# воспользуемся чистым датафреймом
file_path = 'output_dataframe_cleaned.csv'
df = pd.read_csv(file_path)

In [16]:
# посмотрим на общую инфу
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62646 entries, 0 to 62645
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   publish_date_year        62646 non-null  int64  
 1   publish_date             62646 non-null  object 
 2   reason                   62646 non-null  object 
 3   customer_code            62646 non-null  float64
 4   customer_inn             62646 non-null  float64
 5   customer_kpp             62646 non-null  float64
 6   supplier_full_name       62646 non-null  object 
 7   supplier_type            62646 non-null  object 
 8   supplier_inn             62646 non-null  object 
 9   supplier_kpp             36362 non-null  float64
 10  founders_inn_role        62646 non-null  object 
 11  purchase_object          58337 non-null  object 
 12  contract_object          55193 non-null  object 
 13  contract_price_currency  55459 non-null  object 
 14  contract_price_amount 

#### Скорректируем типы данных

In [5]:
# изменим на string

df['publish_date'] = df['publish_date'].astype('string')
df['reason'] = df['reason'].astype('string')
df['customer_code'] = df['customer_code'].astype('string')
df['customer_inn'] = df['customer_inn'].astype('string')
df['customer_kpp'] = df['customer_kpp'].astype('string')
df['supplier_full_name'] = df['supplier_full_name'].astype('string')
df['supplier_type'] = df['supplier_type'].astype('string')
df['supplier_inn'] = df['supplier_inn'].astype('string')
df['supplier_kpp'] = df['supplier_kpp'].astype('string')
df['purchase_object'] = df['purchase_object'].astype('string')
df['contract_object'] = df['contract_object'].astype('string')
df['contract_price_currency'] = df['contract_price_currency'].astype('string')
df['purchase_object'] = df['purchase_object'].astype('string')

In [6]:
# избавимся от .0 в некоторых столбцах

df['customer_code'] = df['customer_code'].str.split('.').str[0]
df['customer_inn'] = df['customer_inn'].str.split('.').str[0]
df['customer_kpp'] = df['customer_kpp'].str.split('.').str[0]
df['supplier_kpp'] = df['supplier_kpp'].str.split('.').str[0]

In [8]:
df.to_csv('output_dataframe_cleaned_correct_datatype.csv', index=False)

> Первичный "сырой" датасет загружен в репозиторий